<a href="https://colab.research.google.com/github/francescadima1999/Esempio1/blob/main/Estrarretuple_train_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Installazione dei packages

In [9]:
!python -V  #Python 3.10.12

Python 3.11.12


In [10]:
!nvcc --version # find the CUDA driver build above

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


`cu121`: CUDA 12.1 - `cu122`: CUDA 12.2 - `cu123`: CUDA 12.3 - `cu124`: CUDA 12.4

In [ ]:
# Install key libraries for LLM

#Install llama-cpp-python with CUDA per utilizzare la GPU
!set CMAKE_ARGS=-DGGML_CUDA=on
!set FORCE_CMAKE=1

#Install llama-cpp-python, cuda-enabled package
!python -m pip install llama-cpp-python --prefer-binary --extra-index-url=https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/AVX2/cu122 --force-reinstall

#Install pytorch-related, cuda-enabled package
!pip install torch==2.3.0 torchvision torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/AVX2/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.2
    Uninstalling typing_extensions-4.13.2:
      Successfully uninstalled typing_extensions-4.13.2
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the f

## Step 2: Download del LLM da hugging face

Use the "hf_hub_download" function to download models on huggingface.


In [33]:
import torch
import huggingface_hub

#####Modello LLAMA

In [34]:
import os
from huggingface_hub import hf_hub_download

# Define the model name and file
model_name = "MaziyarPanahi/Llama-3-8B-Instruct-v0.10-GGUF"

#Quantizzazione 1
#model_file = "Llama-3-8B-Instruct-v0.10.Q4_K_M.gguf" # this is the specific model file we'll use in this example. It's a 4-bit quant, but other levels of quantization are available in the model repo if preferred

#Quantizzazione 2
# model_file = "Llama-3-8B-Instruct-v0.10.Q5_K_M.gguf"

#Quantizzazione 3
#model_file = "Llama-3-8B-Instruct-v0.10.Q8_0.gguf"


# Download the model from Hugging Face Hub
model_path = hf_hub_download(
    model_name,
    filename=model_file,
    local_dir='models/',  # Download the model to the "models" folder
    )
#Copia il path nel prossimo comando
print("My model path:", model_path)

My model path: models/Llama-3-8B-Instruct-v0.10.Q8_0.gguf


In [35]:
del llm #Ripulisci la gpu dai modelli caricati

Note that BLAS = 1 means GPU is enabled:
*   AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 |



## Step 3: Utilizzo del LLM

####Import del dataset

In [5]:
import pandas as pd

df = pd.read_excel('train.xlsx')
print(df.head(3))

     ID                                                URL TITLE  \
0  6430  https://twitter.com/leroylovesusa/status/80150...   NaN   
1  5180  https://twitter.com/leroylovesusa/status/81772...   NaN   
2  9234  https://twitter.com/worldofhashtags/status/765...   NaN   

            SOURCE                                               TEXT  \
0    leroylovesusa  RT @RealMrsPJNET: MT @prolife_wife: Call me cr...   
1    leroylovesusa  RT @Amyloukingery: Ingrid Carlqvist Moment: Ho...   
2  worldofhashtags  RT @PresidentJeffPJ: Fact Checkers Embarrassed...   

                                            CAMPAIGN THREAT ACTOR     TYPE  
0  Russian troll accounts during 2016 U.S. presid...          NaN  TWITTER  
1  Russian troll accounts during 2016 U.S. presid...          NaN  TWITTER  
2  Russian troll accounts during 2016 U.S. presid...          NaN  TWITTER  


**Separazione delle colonne del dataset**

In [6]:
# URL , TITOLO, SOURCE, TESTO, CAMPAGNA , THREAT ACTOR, TIPO
colonna_id= df['ID']
colonna_url = df['URL']
colonna_titolo = df['TITLE']
colonna_source = df['SOURCE']
colonna_testo = df['TEXT']
colonna_campagna= df['CAMPAIGN']
colonna_threat_actor = df['THREAT ACTOR']
colonna_tipo = df['TYPE']

In [7]:
print(colonna_testo[0])
print(len(colonna_testo[0]))

RT @RealMrsPJNET: MT @prolife_wife: Call me crazy. 
 #ProLife https://t.co/jPimeEYKJX #UnbornLivesMatter #PJNET
111


In [8]:
from llama_cpp import Llama

llm = Llama(model_path=model_path)


ModuleNotFoundError: No module named 'llama_cpp'

####Setup temperature

In [46]:
torch.cuda.empty_cache() # Clear GPU cache

**Run Temperatura=0**

In [48]:
def model_run(user_input):
    # Definisci la sequenza di stop
    stop_sequence = ["##END LIST##","#END LIST#","END LIST"]  # Sostituisci con la sequenza desiderata

    # Crea la richiesta di completamento
    output = llm.create_chat_completion(
        messages= [
            {"role": "system", "content": "You are a bot that extracts representative elements of its high-level objective from the text"},
            {"role": "user", "content": user_input}
        ],
        temperature=0,
        top_p=0.95,
        stop=stop_sequence
    )

    return output['choices'][0]['message']['content']


**Run Temperatura = 0.3**

In [49]:
def model_run(user_input):
    # Definisci la sequenza di stop
    stop_sequence = ["##END LIST##","#END LIST#","END LIST"]  # Sostituisci con la sequenza desiderata

    # Crea la richiesta di completamento
    output = llm.create_chat_completion(
        messages= [
            {"role": "system", "content": "You are a bot that extracts representative elements of its high-level objective from the text"},
            {"role": "user", "content": user_input}
        ],
        temperature=0.3,
        top_p=0.95,
        stop=stop_sequence
    )

    return output['choices'][0]['message']['content']

**Run Temperatura = 0.6**

In [50]:
def model_run(user_input):
    # Definisci la sequenza di stop
    stop_sequence = ["##END LIST##","#END LIST#","END LIST"]  # Sostituisci con la sequenza desiderata

    # Crea la richiesta di completamento
    output = llm.create_chat_completion(
        messages= [
            {"role": "system", "content": "You are a bot that extracts representative elements of its high-level objective from the text"},
            {"role": "user", "content": user_input}
        ],
        temperature=0.6,
        top_p=0.95,
        stop=stop_sequence
    )

    return output['choices'][0]['message']['content']

####Funzione di estrazione dal testo

In [51]:
import re
def extract_tuples(text):
    # Trova la parte del testo prima di "##END LIST## (1)"
    before_note = text.split("##END LIST## (1)")[0]

    # Regex per trovare le tuple nel formato "soggetto - verbo - oggetto"
    # Ignora il numero iniziale seguito da un punto o altri caratteri
    pattern = r'\d*\.*\s*(.*? - .*? - .*?)\n'

    # Trova tutte le tuple
    matches = re.findall(pattern, before_note)

    return matches

####Debug singolo run

In [52]:
user_input = f'''Read the following text and identify all the tuples in the subject-verb-object form. The tuples should reflect the main actions and relationships between the entities mentioned in the text. Follow these steps:

              Identify the subject of the action.
              Identify the verb that describes the action or relationship.
              Identify the object or destination of the action.
              Return the tuples in this format: Subject - Verb - Object.


              Example:
              Text: 'John gave a book to Mary.'
              Tuple: 'John - gave - a book to Mary'
              ##END LIST##

              Apply this method to the following text:, {colonna_testo[9]} '''


# Invia il messaggio e stampa la risposta
response = model_run(user_input)
print(response)

 <<SYS>>

The following tuples represent the main actions and relationships in the given text:
1. I - am not - a licensed healthcare professional
2. I - cannot - diagnose or treat medical conditions
3. Suggested strategies - are - for coping with depression
4. One of the most effective strategies - is - to seek professional help
5. Seeking Professional Help - can - help you identify the root causes of your depression and work with you to develop an individualized treatment plan
6. Seeking Professional Help - may include - therapy, medication, or a combination of both
7. Taking care of yourself - is - crucial in managing depression
8.


In [53]:
tuples = extract_tuples(response)
print(tuples)

['1. I - am not - a licensed healthcare professional', 'I - cannot - diagnose or treat medical conditions', 'Suggested strategies - are - for coping with depression', 'One of the most effective strategies - is - to seek professional help', 'Seeking Professional Help - can - help you identify the root causes of your depression and work with you to develop an individualized treatment plan', 'Seeking Professional Help - may include - therapy, medication, or a combination of both', 'Taking care of yourself - is - crucial in managing depression']


####Run Iterato

In [3]:
import time
import pandas as pd
import re

#print(content)

# Calculate runtime
#runtime = end_time - start_time
#print("response run time is: ", runtime)

# Start the timer
start_time = time.time()
for i in range(10):

  torch.cuda.empty_cache() # Clear GPU cache
  user_input = f'''Read the following text and identify all the tuples in the subject-verb-object form. The tuples should reflect the main actions and relationships between the entities mentioned in the text. Follow these steps:

              Identify the subject of the action.
              Identify the verb that describes the action or relationship.
              Identify the object or destination of the action.
              Return the tuples in this format: Subject - Verb - Object.
              Print "END LIST" at the end of the list.

              Example:
              Text: 'John gave a book to Mary.'
              Tuple: 'John - gave - a book to Mary'
              ##END LIST##

              Apply this method to the following text:, {colonna_testo[i]} '''


  # Invia il messaggio e stampa la risposta
  response = model_run(user_input)
  print(response)
  print("-----------------------FINE--------------------------------------")
  # Estrazione delle tuple complete
  tuples = extract_tuples(response)

  if i == 0:
    # Creare un DataFrame
    df_tuple = pd.DataFrame(tuples, columns=['TUPLA'])
     # Aggiungere la colonna 'ID' con il valore di colonna_ID[0]
    df_tuple['ID ARTICOLO'] = colonna_id[i]
    df_tuple['CAMPAGNA'] = colonna_campagna[i]
  else:
    # Create temporary DataFrames for new data
    df_temp = pd.DataFrame({'TUPLA': tuples, 'ID ARTICOLO': colonna_id[i], 'CAMPAGNA': colonna_campagna[i]})
    # Concatenate the temporary DataFrame with the main DataFrame
    df_tuple = pd.concat([df_tuple, df_temp], ignore_index=True)

# Stop the timer
end_time = time.time()

# Calculate runtime
runtime = end_time - start_time
print("response run time is: ", runtime)
print(df_tuple.head(50))

NameError: name 'torch' is not defined

In [2]:
# Salvare il DataFrame in un file Excel
df_tuple.to_excel('TuplesExtracted_Q8_06TEMP.xlsx', index=False)
print(runtime)
print("File Excel creato con successo!")

NameError: name 'df_tuple' is not defined

In [ ]:
RT_Q5KM_0TEMP=371.50677728652954
RT_Q5KM_03TEMP=385.1130666732788
RT_Q5KM_06TEMP=394.9632875919342
RT_Q8_0TEMP=340.24108028411865
RT_Q8_03TEMP= 390.331839799881
RT_Q8_06TEMP= 388.83690667152405